In [2]:
pip install ..

Processing /home/wasit/github/squishy
  Preparing metadata (setup.py) ... done
  Created wheel for monadsquishy: filename=monadsquishy-0.1.4-py3-none-any.whl size=6884 sha256=954b63ebbf1a0df75e55690d95f117bf0bbc232cb176f4864764956b9a2816a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-786stz6n/wheels/ae/e8/3f/fb183d7296e69b421a722aed08c35ba18dd37e0f7d76b1d235
Successfully built monadsquishy
  Attempting uninstall: monadsquishy
    Found existing installation: monadsquishy 0.1.3
    Uninstalling monadsquishy-0.1.3:
      Successfully uninstalled monadsquishy-0.1.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
from monadsquishy import Squishy, sf

In [4]:
import pandas as pd
df = pd.read_parquet('./simple.parquet')

In [13]:

sq_config = {
    'transformations':[
        {
            'input_table': df,
            'transformed_path':'./staging/test1',
            'exploded_path':'./staging/test1',
            'out_columns': {
                 'country_output': {
                     'input':'country',
                     'funcs':[sf.country1,sf.country2,sf.country3]
                 },
                 'name': {
                     'input':'name',
                     'funcs':[lambda x:x, ],
                 },
                 'order_date': {
                     'input':'order_date',
                     'funcs':[sf.date1, sf.date2]
                 },
                 'quantity': {
                     'input':'quantity',
                     'funcs':[sf.quantity1, ]
                 },
                 'price_number': {
                     'input':'price',
                     'funcs':[sf.price1, ]
                 },
                 'price_currency': {
                     'input':'price',
                     'funcs':[sf.currency1, ]
                 } 
             }
        }
    ]
}

In [14]:
sq=Squishy(sq_config)
all_transformed, all_exploded = sq.run()

input:country    -->['country1', 'country2', 'country3']    --> output:country_output
input:name       -->['<lambda>']                            --> output:name
input:order_date -->['date1', 'date2']                      --> output:order_date
input:quantity   -->['quantity1']                           --> output:quantity
input:price      -->['price1']                              --> output:price_number
input:price      -->['currency1']                           --> output:price_currency


In [15]:
sq.input()

,country,name,order_date,quantity,price
0,TH,มานี,9/25/2024,5 ชิ้น,$25.99
1,United States,John Smith,20240926,7 units,30.00 USD
2,ไทย,ปิติ,27-09-2024,pcs: 4,"1,025.99 THB"
3,อังกฤษ,"Smith, J.","Sep 28, 2024",10,"1,000,025.99 บาท"
4,invalid name,J.S.,29 ก.ย. 2567,5pc,๒๗.๙๙ บาท


In [16]:
sq.output()

,country_output,name,order_date,quantity,price_number,price_currency
0,TH,มานี,2024-09-25,5,25.99,USD
1,USA,John Smith,2024-09-26,7,30.00,USD
2,TH,ปิติ,2024-09-27,4,1025.99,THB
3,UK,"Smith, J.",2024-09-28,10,1000025.99,THB
4,None,J.S.,2024-09-29,5,27.99,THB


In [17]:
sq.log()

,input_row,input_column,out_column,input_value,output_value,is_passed,message
0,0,country,country_output,TH,TH,True,Passed: country1()
1,1,country,country_output,United States,USA,False,Failed: country1(): Invalid code
2,1,country,country_output,United States,USA,True,Passed: country2()
3,2,country,country_output,ไทย,TH,False,Failed: country1(): Invalid code
4,2,country,country_output,ไทย,TH,False,Failed: country2(): 'ไทย'
5,2,country,country_output,ไทย,TH,True,Passed: country3()
6,3,country,country_output,อังกฤษ,UK,False,Failed: country1(): Invalid code
7,3,country,country_output,อังกฤษ,UK,False,Failed: country2(): 'อังกฤษ'
8,3,country,country_output,อังกฤษ,UK,True,Passed: country3()
9,4,country,country_output,invalid name,None,False,Failed: country1(): Invalid code
